# Week 4: Advanced OOP - Context Managers, Iterables, ABC & Best Practices

## Learning Objectives
By the end of this week, you should be able to:
- Create and use context managers with `__enter__` and `__exit__`
- Make custom iterable objects with `__iter__` and `__next__`
- Define abstract base classes to create interfaces
- Choose between composition and inheritance
- Use type hints for better code documentation
- Apply dataclasses to reduce boilerplate code
- Debug common OOP issues

**Time:** ~3 hours + bonus exercises

---

## Part 1: Context Managers

### What are Context Managers?

**Context managers** handle **resource management** automatically - they set up resources and clean them up, even if errors occur.

**You've already used context managers:**
```python
with open('data.txt', 'r') as file:
    content = file.read()
# File automatically closed, even if error occurs!
```

**The `with` statement:**
- **Guarantees cleanup:** Resources are released even if code crashes
- **Cleaner code:** No need for manual cleanup
- **Safer:** Prevents resource leaks

---

### Creating Your Own Context Manager

**Two magic methods required:**

**1. `__enter__(self)`:**
- Called when entering the `with` block
- Sets up the resource
- Returns the resource to use in the `with` block

**2. `__exit__(self, exc_type, exc_value, exc_traceback)`:**
- Called when exiting the `with` block (always!)
- Cleans up the resource
- Receives error information if an exception occurred

**The three `exc_*` parameters explained:**
- **`exc_type`:** Type of exception (e.g., `ValueError`, `FileNotFoundError`)
  - `None` if no error occurred
- **`exc_value`:** The actual exception object with the error message
  - `None` if no error occurred
- **`exc_traceback`:** Stack trace showing where the error occurred
  - `None` if no error occurred

**Example:**
```python
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self  # Can return anything to use in 'with'
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = time.time()
        print(f"Time elapsed: {self.end - self.start:.2f}s")
        # Return False to propagate exceptions (default)
        # Return True to suppress exceptions (rare!)
        return False

with Timer():
    # Do something
    time.sleep(1)
# Prints: "Time elapsed: 1.00s"
```

**What is "acquiring/releasing locks"?**
- Think of lab equipment: only one person can use the microscope at a time
- **Acquiring a lock** = checking if available and claiming it ("I'm using this now")
- **Releasing a lock** = marking as available for next person ("Done, someone else can use it")
- Prevents conflicts when multiple people/processes need the same resource

---

### Exercise 4.1.1: Lab Equipment Manager

**Cross-Disciplinary Context:**

**For All Students:**

In any scientific lab, equipment needs proper management:
- **Before use:** Check if available, turn on, calibrate
- **During use:** Record usage, collect data
- **After use:** Turn off, clean, mark as available

**Examples of lab equipment:**
- **Biology:** Microscope, incubator, centrifuge, PCR machine
- **Chemistry:** Spectrophotometer, pH meter, analytical balance
- **Physics:** Oscilloscope, laser, detector

**Why context managers are perfect for this:**
- Ensure equipment is properly initialized
- Guarantee cleanup even if experiment fails
- Track usage automatically
- Prevent equipment damage from improper shutdown

**Real-world scenario:**
Imagine a shared microscope. You must:
1. Check if it's available (acquire)
2. Turn on the light, adjust focus
3. Make observations
4. Turn off light, clean lenses (release)
5. Mark as available for next user

If you forget step 4-5, the next person can't use it!

---

**Your task:** Create a `LabEquipment` context manager.

**Requirements:**

**Attributes:**
- `name` (str): Equipment name (e.g., "Microscope A")
- `is_available` (bool): Whether equipment is free to use
- `usage_count` (int): Number of times used

**Methods:**

1. **`__enter__(self)`:**
   - Check if `self.is_available` is True
   - If not available, raise `RuntimeError(f"{self.name} is currently in use")`
   - If available:
     - Set `self.is_available = False` (acquire the lock)
     - Increment `self.usage_count`
     - Print: `"Starting {self.name}"`
     - Return `self`

2. **`__exit__(self, exc_type, exc_value, exc_traceback)`:**
   - Set `self.is_available = True` (release the lock)
   - If `exc_type is None` (no error):
     - Print: `"Shutting down {self.name} normally"`
   - Else (error occurred):
     - Print: `"Emergency shutdown of {self.name} due to {exc_type.__name__}"`
   - Return `False` (don't suppress exceptions)

**Note:** The equipment should be released even if an error occurs during use!

In [ ]:
class LabEquipment:
    """Context manager for lab equipment."""
    
    def __init__(self, name):
        self.name = name
        self.is_available = True
        self.usage_count = 0
    
    # Your code here:
    # 1. Implement __enter__(self)
    # 2. Implement __exit__(self, exc_type, exc_value, exc_traceback)


# Test your LabEquipment context manager
microscope = LabEquipment("Microscope A")

print("=== Normal use ===")
with microscope:
    print("Using microscope to observe cells...")
    print(f"Available: {microscope.is_available}")  # Should be False

print(f"Available after use: {microscope.is_available}")  # Should be True
print(f"Usage count: {microscope.usage_count}")

print("\n=== Use with error ===")
try:
    with microscope:
        print("Using microscope...")
        raise ValueError("Sample slide broke!")
except ValueError:
    print("Error was caught")

print(f"Available after error: {microscope.is_available}")  # Should be True!
print(f"Usage count: {microscope.usage_count}")

---

## Part 2: Iterables and Iterators

### What is an Iterable?

**Iterable:** Any object you can loop through with a `for` loop.

**Examples you already know:**
```python
for item in [1, 2, 3]:        # List is iterable
for char in "hello":           # String is iterable
for key in {"a": 1, "b": 2}:   # Dictionary is iterable
for num in range(5):           # range is iterable
```

**How does it work?**
When Python sees `for item in my_object:`, it:
1. Calls `iter(my_object)` to get an **iterator**
2. Repeatedly calls `next(iterator)` to get items
3. Stops when `StopIteration` exception is raised

---

### Creating Your Own Iterable

**Two magic methods required:**

**1. `__iter__(self)`:**
- Returns an **iterator** object
- Usually returns `self` (if the class is its own iterator)
- Should reset iteration state if necessary

**2. `__next__(self)`:**
- Returns the next item
- Raises `StopIteration` when there are no more items
- Keeps track of current position

**Example:**
```python
class Countdown:
    def __init__(self, start):
        self.start = start
        self.current = start
    
    def __iter__(self):
        self.current = self.start  # Reset
        return self  # This object is its own iterator
    
    def __next__(self):
        if self.current <= 0:
            raise StopIteration  # Done!
        
        value = self.current
        self.current -= 1
        return value

# Use it:
for num in Countdown(5):
    print(num)  # Prints: 5, 4, 3, 2, 1
```

**Key points:**
- `__iter__` resets state and returns self
- `__next__` returns next item or raises `StopIteration`
- `StopIteration` is not an error - it's how iteration ends!

---

### Exercise 4.2.1: DNA Sequence Iterator

**Biology Background:**

**For Chemistry/Physics Students:**

**DNA (Deoxyribonucleic Acid)** stores genetic information:

**Structure:**
- Made of four nucleotides (building blocks):
  - **A** (Adenine)
  - **T** (Thymine)
  - **G** (Guanine)
  - **C** (Cytosine)
- DNA is a long sequence of these letters: `"ATGCGATCG..."`

**Codons:**
- DNA is read in groups of **3 nucleotides** called **codons**
- Each codon codes for one amino acid
- Example: `"ATGCGATCG"` → codons are `"ATG"`, `"CGA"`, `"TCG"`

**Why this matters:**
- Codons determine which proteins are made
- Mutations (changes in DNA) can change codons
- Understanding codons helps with:
  - Gene analysis
  - Protein prediction
  - Disease research

**Why use an iterator:**
- DNA sequences can be millions of bases long
- Processing in chunks (codons) is more efficient
- Can stop early if you find what you're looking for

---

**Your task:** Create a `DNASequence` iterable that yields codons.

**Requirements:**

**Attributes:**
- `sequence` (str): The DNA sequence (e.g., "ATGCGATCG")
- `position` (int): Current position in the sequence

**Methods:**

1. **`__init__(self, sequence)`:**
   - Store the sequence
   - Initialize position to 0
   - Validate: sequence should only contain A, T, G, C
     - If invalid characters found, raise `ValueError`

2. **`__iter__(self)`:**
   - Reset `self.position` to 0
   - Return `self`

3. **`__next__(self)`:**
   - If `self.position >= len(self.sequence)`, raise `StopIteration`
   - Get the next 3 characters: `codon = self.sequence[self.position:self.position+3]`
   - If codon has fewer than 3 characters, raise `StopIteration` (incomplete codon)
   - Increment `self.position` by 3
   - Return the codon

4. **`__len__(self)`:** (bonus magic method)
   - Return the number of complete codons: `len(self.sequence) // 3`

In [ ]:
class DNASequence:
    """Iterable DNA sequence that yields codons (3-base groups)."""
    
    # Your code here:
    # 1. Implement __init__(self, sequence) with validation
    # 2. Implement __iter__(self)
    # 3. Implement __next__(self)
    # 4. Implement __len__(self) - bonus


# Test your DNASequence iterator
dna = DNASequence("ATGCGATCGAAATTT")

print("=== Iterating over codons ===")
for codon in dna:
    print(codon)

print(f"\nNumber of complete codons: {len(dna)}")

print("\n=== Can iterate multiple times ===")
codons = list(dna)
print(f"Codons: {codons}")

print("\n=== Validation test ===")
try:
    invalid_dna = DNASequence("ATGXYZ")  # Invalid characters
except ValueError as e:
    print(f"Error caught: {e}")

---

## Part 3: Abstract Base Classes (ABC) 

### What are Abstract Base Classes?

**Abstract Base Class (ABC):** A class that defines an **interface** (contract) that child classes must follow.

**Key concept:** ABCs define **what methods must exist**, but not **how they work**.

**Why use ABCs:**
- **Enforce interface:** Guarantee all child classes have required methods
- **Documentation:** Clearly show what methods are needed
- **Catch errors early:** Get error at class definition, not at runtime
- **Design clarity:** Separate interface from implementation

**Important:** An ABC by itself doesn't do anything - it's just a template!
- You **cannot create instances** of an ABC
- Child classes **must implement** all abstract methods
- If they don't, Python raises an error when you try to create an instance

---

### Creating Abstract Base Classes

**Three steps:**

1. **Import ABC tools:**
```python
from abc import ABC, abstractmethod
```

2. **Inherit from ABC:**
```python
class Shape(ABC):
```

3. **Mark required methods with @abstractmethod:**
```python
@abstractmethod
def area(self):
    pass  # No implementation - child must provide!
```

**Example:**
```python
from abc import ABC, abstractmethod

class Shape(ABC):
    """Abstract base class for shapes."""
    
    @abstractmethod
    def area(self):
        """Calculate area. Must be implemented by child classes."""
        pass
    
    @abstractmethod
    def perimeter(self):
        """Calculate perimeter. Must be implemented by child classes."""
        pass

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14159 * self.radius ** 2
    
    def perimeter(self):
        return 2 * 3.14159 * self.radius

# This works:
c = Circle(5)

# This fails:
# s = Shape()  # TypeError: Can't instantiate abstract class
```

**When to use ABCs:**
- You have multiple classes that should share an interface
- You want to guarantee specific methods exist
- You're designing a framework or library
- You want to use polymorphism safely

---

### Exercise 4.3.1: Cell Organelles

**Biology Background:**


**Cell organelles** are specialized structures inside cells (like organs in your body):

**Common organelles:**

1. **Mitochondrion** (plural: mitochondria)
   - **Function:** Produces energy (ATP) for the cell
   - **Nickname:** "Powerhouse of the cell"
   - **Process:** Cellular respiration (glucose + oxygen → ATP)
   - **Found in:** All eukaryotic cells (animals, plants, fungi)

2. **Nucleus**
   - **Function:** Stores and protects DNA
   - **Nickname:** "Control center"
   - **Process:** Controls cell activities, gene expression
   - **Found in:** All eukaryotic cells

3. **Ribosome**
   - **Function:** Protein synthesis (makes proteins)
   - **Process:** Translates mRNA into proteins
   - **Found in:** All cells (even bacteria)

4. **Chloroplast** (plants only)
   - **Function:** Photosynthesis (makes glucose from sunlight)
   - **Process:** Light + CO₂ + H₂O → glucose + O₂
   - **Found in:** Plant cells and algae

**Why ABC is perfect here:**
- All organelles have a function (what they do)
- All should be able to describe themselves
- ABC enforces: "If you claim to be an Organelle, you MUST implement these methods"
- Prevents incomplete organelle classes

**Real-world analogy:**
- ABC = Job description ("All employees must have ID badge and job function")
- Derived classes = Specific employees (each has ID badge + specific role)

---

**Your task:** Create an `Organelle` ABC with specific organelle implementations.

**Requirements:**

**1. Abstract Base Class: `Organelle`**

- Inherit from `ABC`
- Abstract method: `get_function()` - returns string describing function
- Abstract method: `__str__()` - returns string representation
  - This ensures all organelles can be printed nicely!

**2. Concrete Class: `Mitochondrion(Organelle)`**

- Attribute: `atp_production_rate` (float) - ATP molecules per second
- Implement `get_function()`: Returns `"Produces ATP through cellular respiration"`
- Implement `__str__()`: Returns `f"Mitochondrion (ATP rate: {self.atp_production_rate}/s)"`

**3. Concrete Class: `Nucleus(Organelle)`**

- Attribute: `chromosomes` (int) - number of chromosomes
- Implement `get_function()`: Returns `"Stores and protects genetic material (DNA)"`
- Implement `__str__()`: Returns `f"Nucleus ({self.chromosomes} chromosomes)"`

**4. Function: `describe_organelles(organelle_list)`**

- Takes a list of Organelle objects
- For each organelle, prints:
  - The organelle (using `str()`)
  - Its function (using `get_function()`)
- This demonstrates polymorphism - same code works for all organelle types!

In [ ]:
from abc import ABC, abstractmethod

# Your code here:
# 1. Create Organelle ABC with abstract methods
# 2. Create Mitochondrion class
# 3. Create Nucleus class
# 4. Create describe_organelles function


# Test your classes
mito = Mitochondrion(atp_production_rate=1000.0)
nucleus = Nucleus(chromosomes=46)

print("=== Individual organelles ===")
print(mito)
print(f"Function: {mito.get_function()}")
print()
print(nucleus)
print(f"Function: {nucleus.get_function()}")

print("\n=== Polymorphism in action ===")
organelles = [mito, nucleus]
describe_organelles(organelles)

print("\n=== Cannot instantiate ABC ===")
try:
    org = Organelle()  # Should fail!
except TypeError as e:
    print(f"Error: {e}")

---

## Part 5: Type Hints and Dataclasses

### Type Hints

**Type hints** are annotations that show what types variables/parameters should be.

**Important: Type hints do NOT affect code execution!**
- Python doesn't enforce types
- Code runs the same with or without type hints
- They're for **documentation** and **tooling** (IDEs, linters)

**Basic syntax:**
```python
def greet(name: str) -> str:
    return f"Hello, {name}"

age: int = 25
height: float = 1.75
is_student: bool = True
```

**Why use type hints:**
- **Better IDE support:** Autocomplete, error detection
- **Documentation:** Shows what types are expected
- **Catch bugs early:** Linters can find type mismatches
- **Code clarity:** Makes intentions explicit

**Common type hints:**
- `int`, `float`, `str`, `bool`
- `list[int]` - list of integers
- `dict[str, float]` - dictionary with string keys, float values
- `Optional[str]` - can be str or None
- `Union[int, float]` - can be int or float

---

### Dataclasses

**Dataclass** = Class decorator that auto-generates common methods.

**What @dataclass generates:**
- `__init__()` - constructor from attributes
- `__repr__()` - string representation
- `__eq__()` - equality comparison
- And more!

**Without dataclass:**
```python
class Sample:
    def __init__(self, id: str, temp: float, ph: float):
        self.id = id
        self.temp = temp
        self.ph = ph
    
    def __repr__(self):
        return f"Sample(id={self.id}, temp={self.temp}, ph={self.ph})"
    
    def __eq__(self, other):
        return (self.id == other.id and 
                self.temp == other.temp and 
                self.ph == other.ph)
```

**With dataclass:**
```python
from dataclasses import dataclass

@dataclass
class Sample:
    id: str
    temp: float
    ph: float
```

**Much less code, same functionality!**

---

### Exercise 4.5.1: Experimental Data with Type Hints and Dataclass

**Cross-Disciplinary Context:**

**For All Students:**

In any scientific experiment, you collect data:
- **Sample ID:** Unique identifier
- **Measurements:** Temperature, pH, concentration, absorbance, etc.
- **Timestamp:** When measured
- **Valid:** Whether data passed quality checks

**Examples:**
- **Biology:** Cell count, viability, growth rate
- **Chemistry:** Concentration, absorbance, reaction rate
- **Physics:** Voltage, current, resistance, frequency

**Why dataclasses are perfect:**
- Lots of attributes to store
- Need clear types (int, float, str, bool)
- Want automatic `__repr__` for logging
- Want automatic `__eq__` for comparison
- Less boilerplate = fewer bugs!

---

**Your task:** Create experimental data classes with type hints.

**Part A: Regular class with type hints**

Create `Measurement` class (WITHOUT @dataclass):
- Attribute `value` (float): The measured value
- Attribute `unit` (str): Unit of measurement
- Attribute `error` (float): Measurement uncertainty
- Method `__init__` with type hints
- Method `__repr__` that returns: `f"Measurement({self.value} ± {self.error} {self.unit})"`

**Part B: Dataclass**

Create `ExperimentalData` dataclass (WITH @dataclass):
- `sample_id` (str): Sample identifier
- `temperature` (float): Temperature in Celsius
- `ph` (float): pH value
- `concentration` (float): Concentration in M
- `is_valid` (bool): Whether data passed QC

No need to write `__init__` or `__repr__` - @dataclass generates them!

In [ ]:
from dataclasses import dataclass

# Part A: Regular class with type hints
class Measurement:
    """A measurement with value, unit, and error."""
    
    # Your code here:
    # 1. Implement __init__ with type hints
    # 2. Implement __repr__


# Part B: Dataclass (auto-generates __init__, __repr__, __eq__)
# Your code here:
# Create ExperimentalData dataclass with type hints


# Test Measurement class
print("=== Measurement class ===")
m1 = Measurement(25.3, "°C", 0.1)
print(m1)

# Test ExperimentalData dataclass
print("\n=== ExperimentalData dataclass ===")
data1 = ExperimentalData(
    sample_id="EXP001",
    temperature=25.0,
    ph=7.4,
    concentration=0.1,
    is_valid=True
)
print(data1)  # Auto-generated __repr__!

data2 = ExperimentalData("EXP002", 25.0, 7.4, 0.1, True)
print(f"\ndata1 == data2: {data1 == data2}")  # Auto-generated __eq__!

---

## Part 6: Debugging Common OOP Issues

### Understanding Mutable vs Immutable

**This is crucial for avoiding bugs!**

**Immutable** (cannot change after creation):
- `int`, `float`, `str`, `tuple`, `bool`
- Example: When you do `x = 5; x = 6`, you create a NEW object

**Mutable** (can change after creation):
- `list`, `dict`, `set`
- Example: `lst = [1,2]; lst.append(3)` modifies the SAME object

**Why this matters:**

**The dangerous default argument bug:**
```python
# WRONG - Bug!
class Experiment:
    def __init__(self, samples=[]):  # ❌ Mutable default!
        self.samples = samples

exp1 = Experiment()
exp1.samples.append("A")

exp2 = Experiment()  # Expects empty list
print(exp2.samples)  # ["A"] - WRONG! Shares list with exp1!

# CORRECT
class Experiment:
    def __init__(self, samples=None):  # ✓ Use None
        if samples is None:
            self.samples = []  # New list each time
        else:
            self.samples = samples
```

**Why:** Default arguments are evaluated ONCE when function is defined!
- Same list object reused for all instances
- Changes affect all instances

---

### What is Hashable?

**Hashable** = Can be used as dictionary key or set member.

**Rules:**
- Must be **immutable**
- Has a `__hash__()` method that returns consistent value

**Hashable types:**
- ✓ `int`, `float`, `str`, `tuple` (of hashable items)

**NOT hashable:**
- ❌ `list`, `dict`, `set`

**Examples:**
```python
# Works - strings are hashable
data = {"sample_1": 25, "sample_2": 30}

# Works - tuples are hashable
coords = {(0, 0): "origin", (1, 1): "point"}

# Error - lists are not hashable
data = {[1, 2]: "value"}  # TypeError!
```

**Why:** Hash value must not change, so object must be immutable!

---

### Exercise 4.6.1: Fix the Bugs

**Your task:** Identify and fix common OOP bugs.

**Three bugs to fix:**

In [ ]:
# Bug 1: Mutable default argument
# What's wrong with this code?

class SampleSet:
    def __init__(self, samples=[]):
        self.samples = samples
    
    def add_sample(self, sample):
        self.samples.append(sample)

# Test it - notice the bug!
set1 = SampleSet()
set1.add_sample("A")
print(f"Set 1: {set1.samples}")

set2 = SampleSet()  # Should be empty!
print(f"Set 2: {set2.samples}")  # Bug: shares list with set1!

print("\nYour fixed version:")
# Fix the bug here by modifying __init__

In [ ]:
# Bug 2: Forgetting self in method
# What's wrong with this code?

class Reaction:
    def __init__(self, rate_constant):
        self.rate_constant = rate_constant
    
    def calculate_rate(concentration):  # Missing something!
        return rate_constant * concentration

# This will fail:
# rxn = Reaction(0.5)
# rate = rxn.calculate_rate(2.0)
# TypeError: calculate_rate() takes 1 positional argument but 2 were given

print("Fix the calculate_rate method:")
# Your fixed version here

In [ ]:
# Bug 3: Property decorator order
# What's wrong with this code?

class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius
    
    @celsius.setter  # Wrong order!
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Below absolute zero!")
        self._celsius = value
    
    @property
    def celsius(self):
        return self._celsius

# This will fail with NameError

print("Fix the decorator order:")
# Your fixed version here

---

## BONUS Exercise: Combine Everything!
**Challenge:** Create a complete lab equipment management system using all concepts from this week.

**Requirements:**

1. **ABC:** Create `LabInstrument` abstract base class
   - Abstract method: `calibrate()`
   - Abstract method: `measure(sample)` → returns measurement

2. **Concrete classes:** Create specific instruments
   - `Spectrophotometer(LabInstrument)` - measures absorbance
   - `PHMeter(LabInstrument)` - measures pH

3. **Dataclass:** Create `Measurement` dataclass
   - `value` (float)
   - `unit` (str)
   - `timestamp` (str)
   - `instrument_id` (str)

4. **Context manager:** Make instruments work with `with` statement
   - `__enter__`: calibrate and prepare
   - `__exit__`: shutdown and cleanup

5. **Composition:** Create `Lab` class
   - Has a list of instruments
   - Can add/remove instruments
   - Can process samples through all instruments

6. **Iterator:** Make `Lab` iterable
   - Can iterate through all instruments

**Stretch goal:** Add type hints throughout!

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from datetime import datetime

# Your complete solution here:
# This should combine:
# - ABC (LabInstrument)
# - Inheritance (Spectrophotometer, PHMeter)
# - Dataclass (Measurement)
# - Context managers (__enter__, __exit__)
# - Composition (Lab has instruments)
# - Iterator (Lab is iterable)
# - Type hints (bonus)


# Example usage:
# lab = Lab()
# lab.add_instrument(Spectrophotometer("SPEC-001"))
# lab.add_instrument(PHMeter("PH-001"))
#
# for instrument in lab:
#     with instrument:
#         measurement = instrument.measure(sample="Water")
#         print(measurement)

---

## Summary & Reflection

### Key Concepts Covered

**Part 1: Context Managers**
- `__enter__` and `__exit__` methods
- Resource management and cleanup
- Understanding exc_type, exc_value, exc_traceback
- Acquiring and releasing locks

**Part 2: Iterables and Iterators**
- What "iterable" means
- `__iter__` and `__next__` methods
- StopIteration exception
- Creating custom iteration behavior

**Part 3: Abstract Base Classes**
- Defining interfaces with ABC
- @abstractmethod decorator
- Enforcing method implementation
- When and why to use ABCs

**Part 4: Composition vs Inheritance**
- "Has-a" vs "Is-a" relationships
- When to use composition
- When to use inheritance
- Prefer composition for flexibility

**Part 5: Type Hints and Dataclasses**
- Type hints for documentation (not enforcement)
- @dataclass decorator
- Auto-generated methods
- Reducing boilerplate code

**Part 6: Debugging**
- Mutable vs immutable types
- Mutable default argument bug
- Hashable types for dict keys
- Common OOP mistakes

---

### Reflection Questions

1. **When would you use a context manager vs manual cleanup?**
   - Think about: error handling, resource safety, code clarity

2. **What's the benefit of making a class iterable?**
   - Consider: custom collections, lazy evaluation, memory efficiency

3. **When should you create an Abstract Base Class?**
   - Think about: multiple implementations, interface guarantees, polymorphism

4. **How do you decide between composition and inheritance?**
   - Ask: Is-a or Has-a? Need flexibility? Want to swap components?

5. **Why use type hints if Python doesn't enforce them?**
   - Consider: IDE support, documentation, team collaboration, bug prevention

---

## Next Steps

**Week 5: Midterm Exam + Project Finalization**
- Midterm covers Weeks 1-4
- Project work session
- Final touches on group projects

**Great work completing Week 4!**